<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/DL/Group_3DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning with PyTorch
## 3D Convolutional Neural Network on Group Brain fMRI
Contributors: Stacey Rivet Beck
### To Do:
- Either:
  - Get Raw data in 4D   
          - OR -
  - Reshape Raw data into 4D from 2D
    - Apply Whole Brain Mask to data and save to AWS

- Build Dataloader: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

- Implement 3DCNN from paper: REALLY GREAT PAPERS
  - Nguyen et al. http://proceedings.mlr.press/v136/nguyen20a/nguyen20a.pdf
  - Wang et al. https://arxiv.org/pdf/1801.09858.pdf (discusses more in detail the input data shapes and processing)
  
  - Inputs: 84@ x * y * z ; one fmri time series at a time, not concatenated
  - Basic Architecture:
        #First layer is generating temporal descriptors of the voxels
        Conv1 1 x 1 x 1 filter, output = 32, stride = 1, ReLU, BatchNorm
        Conv2 7 x 7 x 7 filter, output = 64, stride = 2, ReLU, BatchNorm
        Conv3 3 x 3 x 3 fitler, output = 64, stride = 2, ReLU, BatchNorm
        Conv4 3 x 3 x 3 fitler, output = 128, stride = 2, ReLU, BatchNorm
        Global Average Pooling on final feature maps ->
        Flattened maps size 128?
        Fully connected layer size 64
        Fully connected layer size 2 (2 way classification, one for each class) -> softmax

        Optimized with Adam, standard parameters (β1=0.9 and β2=0.999)
        Batched at 32, but we may need to batch smaller due to GPU compute
        Learning Rate = 0.001, gradual decay after Val loss plateaued after 15 epochs
        Cross entropy Loss
        Employ early stopping
        In Wang et al. they used data for visualization, same size as input data, though are reduced in time dimension to be mapped on fsaverage surface. 
        
  

In [ ]:
# Pytroch Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
#import torchvision.transforms as transforms
from torch.nn import ReLU, CrossEntropyLoss, Conv3d, Module, Softmax, AdaptiveAvgPool3d
from torch.optim import Adam, SGD
#from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

## Build Dataloader

## Build Model

In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    
    #Conv1
    self.conv1 = nn.Conv3d(in_channels = 1, 
                           out_channels = 32, 
                           kernel_size = 1, 
                           stride = 1
                           )
    self.bn1 = nn.BatchNorm3d(32)
    self.conv2 = nn.Conv3d(in_channels = 32, 
                           out_channels = 64, 
                           kernel_size = 7, 
                           stride = 2
                           )
    self.bn2 = nn.BatchNorm3d(64)
    self.conv3 = nn.Conv3d(in_channels = 64, 
                           out_channels = 64, 
                           kernel_size = 3, 
                           stride = 2
                           )
    self.bn3 = nn.BatchNorm3d(64)
    self.conv4 = nn.Conv3d(in_channels = 64, 
                           out_channels = 128, 
                           kernel_size = 3, 
                           stride = 2
                           )
    self.bn4 = nn.BatchNorm3d(128) 
    self.pool1 = nn.AdaptiveAvgPool3d((1,1,1)) #Global Average Pool, takes the average over last two dimensions to flatten 
  
                                                             
    # Fully connected layer
    self.fc1 = nn.Linear(128*?*?,64) # need to find out the size where AdaptiveAvgPool 
    self.fc2 = nn.Linear(64, 2) # left with 2 for the two classes                     



  def forward(self, x):
    x = self.bn1(F.relu(self.conv1((x))))
    x = self.bn2(F.relu(self.conv2((x))))
    x = self.bn3(F.relu(self.conv3((x))))
    x = self.pool1(self.bn4(F.relu(self.conv4((x)))))
    x = self.fc1(x)
    x = F.softmax(self.fc2(x))
    return x                        




# Set to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Get model
model = ConvNet()
model = model.to(device)
print("First model training on GPU")
print(model)

# Initialize other parameters
epochs = 25 #120
learning_rate = 0.001
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)#, momentum = 0.9) #or ADAM/ momentum

## Training

In [ ]:
accuracy_stats = {
    'train': [],
    'val': []
  }

print(accuracy_stats)

loss_stats = {
    'train': [],
    'val': []
    }
print(loss_stats)



def train_val_model(epochs):
  for epoch in range(1, epochs + 1):

    # TRAINING *****************************************************************

    train_epoch_loss = 0
    train_epoch_acc = 0

    # set model in training mode 
    model.train()
    print('\nEpoch$ : %d'%epoch)
    for x_train_batch, y_train_batch in tqdm(train_loader):
      x_train_batch = x_train_batch.to(device)#(float).to(device) # for GPU support
      y_train_batch = y_train_batch.to(device) 

      #print(x_train_batch.shape)

      # sets gradients to 0 to prevent interference with previous epoch
      optimizer.zero_grad()
    
      # Forward pass through NN
      y_train_pred = model(x_train_batch)#.to(float)
      train_loss = criterion(y_train_pred, y_train_batch)
      train_acc = accuracy(y_train_pred, y_train_batch)

      # Backward pass, updating weights
      train_loss.backward()
      optimizer.step()

      # Statistics
      train_epoch_loss += train_loss.item()
      train_epoch_acc += train_acc.item()


    # VALIDATION****************************************************************   
    
    with torch.set_grad_enabled(False):
      val_epoch_loss = 0
      val_epoch_acc = 0

      model.eval()
      for x_val_batch, y_val_batch in tqdm(validate_loader):
      
        x_val_batch =  x_val_batch.to(device)#.to(float)
        y_val_batch = y_val_batch.to(device)
            
        # Forward pass
        y_val_pred = model(x_val_batch)#.to(float)   
        val_loss = criterion(y_val_pred, y_val_batch)
        val_acc = accuracy(y_val_pred, y_val_batch)
            
        val_epoch_loss += val_loss.item()
        val_epoch_acc += val_acc.item()

    # Prevent plateauing validation loss 
    #scheduler.step(val_epoch_loss/len(validate_loader))

        
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(validate_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(validate_loader))
                              
    
    print(f'Epoch {epoch+0:03}: Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(validate_loader):.5f}') 
    print(f'Train Acc: {train_epoch_acc/len(train_loader):.3f} | Val Acc: {val_epoch_acc/len(validate_loader):.3f}')

      




def accuracy(y_pred, y_test):
  # Calculating model accuracy at each epoch 
  y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
  _, y_pred_prob = torch.max(y_pred_softmax, dim = 1)
  correct_pred = (y_pred_prob == y_test).float()
  acc = correct_pred.sum() / len(correct_pred)
  acc = torch.round(acc * 100)

  return acc



     





if __name__ == '__main__':
  train_val_model(epochs)